# Project 1: Milestone 2 - White Paper
### DSC680-T301 Applied Data Science
### Joshua Greenert
### 3/27/2023

## Data Preparation

In [181]:
# Set some required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pull in the data.
df_housing = pd.read_csv('Housing.csv')
df_hpi = pd.read_csv('HPI_master.csv')
df_sale_prices_states = pd.read_csv('Sale_Prices_State.csv')

df_state_stats = pd.read_excel('state_statistics_for_download.xls')

In [182]:
# Fix the state stats dataframe.
import datetime

# Correct the row headers for the state stats.
df_state_stats.columns = df_state_stats.iloc[3]

#remove first row from DataFrame
df_state_stats = df_state_stats[4:]

# Remove all values of US from the State column
df_state_stats = df_state_stats[df_state_stats['State'] != 'US']

# Remove the rows with Q2, Q3, and Q4 data.
df_state_stats = df_state_stats[~df_state_stats['Year-Quarter'].str.contains('Q[234]')]

# Loop through the dataframe and replace each year-quarter with an actual date object.
for index, row in df_state_stats.iterrows():
    year = int(row['Year-Quarter'][0:4])
    date = datetime.date(year, 1, 1)
    df_state_stats.at[index,'Year-Quarter'] = date

# Show the head to confirm working
df_state_stats.head(3)

3,State,Year-Quarter,Average Price,Median Price
46,AK,2000-01-01,159887,148406
50,AK,2001-01-01,168500,155637
54,AK,2002-01-01,175754,161283


In [183]:
# Fix the area codes so that they are strings and have 0's at the beginning if the number is less than 5 digits.
for index, row in df_housing.iterrows():
    if(len(str(row['area'])) < 5):
        zipCode = "0" + str(row['area'])
        df_housing.at[index,'area'] = zipCode
    else:
        df_housing.at[index,'area'] = str(row['area'])
        
df_housing.head(3)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,07420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,08960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,09960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished


In [184]:
# Remove all rows with level that is not equal to state.
df_hpi = df_hpi[df_hpi['level'] == 'State']

# Remove all the periods not equal to 1
df_hpi = df_hpi[df_hpi['period'] == 1]

# Update the year to be a date; otherwise visuals won't work right.
for index, row in df_hpi.iterrows():
    year = int(row['yr'])
    date = datetime.date(year, 1, 1)
    df_hpi.at[index,'yr'] = date

# filter out any rows where hpi_flavor is not equal to all-transactions, and traditional data for hpi-type
df_hpi = df_hpi[df_hpi['hpi_flavor'] == "all-transactions"]
df_hpi = df_hpi[df_hpi['hpi_type'] == "traditional"]

# Drop all columns not needed
df_hpi = df_hpi.drop(['period', 'index_sa', 'frequency', 'level', 'hpi_flavor','hpi_type'], axis = 1)

# Show the head to confirm working
df_hpi.head(3)

,place_name,place_id,yr,index_nsa
67915,Alaska,AK,1975-01-01,62.05
67919,Alaska,AK,1976-01-01,71.34
67923,Alaska,AK,1977-01-01,78.24


In [185]:
# Remove all columns besides the state name and most recent pricing column with all values.
df_sale_prices_states = df_sale_prices_states.loc[:, ['RegionName', '2020-02']]

# Create a dict to map over.
state_abbr = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# create new column with state abbreviations
df_sale_prices_states['State'] = df_sale_prices_states['RegionName'].map(state_abbr)

df_sale_prices_states.head(3)

,RegionName,2020-02,State
0,California,524400,CA
1,Texas,239900,TX
2,New York,324800,NY


## Build Models: Linear Regression

In [186]:
# Let's attempt a model based on linear regression using the hpi data
from sklearn.linear_model import LinearRegression

# Extract the year from the yr column
df_hpi["Year"] = pd.to_datetime(df_hpi["yr"]).dt.year

# Convert the State column to one-hot encoding
df_hpi = pd.concat([df_hpi, pd.get_dummies(df_hpi["place_id"])], axis=1)

# Define the input features (all columns except Year-Quarter and Price)
X = df_hpi.drop(["yr", "index_nsa", "place_id", "place_name"], axis=1)

# Define the target variable (index_nsa)
y = df_hpi["index_nsa"]

# Split the data into training and testing sets
X_train = X[X["Year"] < 2015]
y_train = y[X["Year"] < 2015]
X_test = X[X["Year"] >= 2015]
y_test = y[X["Year"] >= 2015]

# Fit the model
model = LinearRegression().fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 13714.800694736376


Linear Regression is not a wise choice for time series data, but I wanted to see how close the predictions could be with this model.  As we can see from the Mean Squared Error (MSE), the model is far away from anything near an accurate prediction.  If the hpi index is based on a 100 point value, then being greater than 13,000 is too far away to ever be valuable.  

## Build Models: ARIMA (Time Series)

In [187]:
# Let's attempt a model based on ARIMA which is more catered towards these values.
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

# Set the index to the year column
df_hpi['yr'] = pd.to_datetime(df_hpi['yr'])
df_hpi.set_index('yr', inplace=True)

# Define the states to make predictions for
states = df_hpi['place_id'].unique()

# Set up an empty dataframe to hold the predictions
predictions_df = pd.DataFrame()

# Loop through each state and make a prediction for 2020-2030
for state in states:
    # Get the data for the current state
    state_df = df_hpi[df_hpi['place_id'] == state]
    
    # Split the data into training and testing sets
    train = state_df.loc[:'2019']
    test = state_df.loc['2020':]
    
    # Define the ARIMA model with a downward trend
    model = sm.tsa.statespace.SARIMAX(train['index_nsa'], order=(1,1,1), trend='c', seasonal_order=(1, 0, 0, 12))
    
    # Fit the model
    results = model.fit()
    
    # Make a forecast for the next 10 years
    forecast = results.get_forecast(steps=11)
    
    # Extract the predicted values
    predicted_values = forecast.predicted_mean
    
    # Add the predicted values to the predictions dataframe
    predictions_df[state] = predicted_values
    
# Add the year column to the predictions dataframe
predictions_df['year'] = pd.date_range(start='2020-01-01', periods=11, freq='Y')

# Set the year column as the index
predictions_df.set_index('year', inplace=True)

# Rename the columns to the state abbreviations
predictions_df.columns = states

In [188]:
predictions_df.head(11)


,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
year,,,,,,,,,,,,,,,,,,,,,
2020-12-31,331.940764,333.740232,293.978544,430.887779,655.642776,589.912605,406.951388,911.050764,473.005989,474.527708,...,393.692021,401.485814,346.314407,513.270682,474.682543,490.507316,655.640590,372.412131,232.197194,318.652745
2021-12-31,337.820871,341.693567,299.857569,440.399142,666.180663,611.622531,417.765893,934.342838,485.871844,481.320240,...,400.083144,412.633438,357.218421,520.994915,485.419657,498.461777,665.218227,383.334745,236.662436,323.211557
2022-12-31,343.672361,347.786928,304.683414,447.344382,678.244742,633.575971,429.483602,955.942596,499.305336,487.612155,...,405.236695,419.456276,366.263280,521.576876,495.657616,506.223932,670.217866,392.672981,241.442345,328.101584
2023-12-31,349.629597,353.509808,309.624998,454.342314,690.797941,652.912228,438.964763,976.596286,511.239133,495.183200,...,411.571353,426.776721,374.562444,526.002067,505.215075,515.086400,677.985917,401.288014,245.892701,333.456005
2024-12-31,355.554059,359.267265,314.628021,462.148691,703.619220,670.043322,448.536677,996.679738,522.514376,503.502432,...,417.918393,434.044599,382.426636,534.400089,514.451411,524.110636,687.083865,409.350115,250.225779,339.028664
2025-12-31,361.552958,364.842484,319.645150,471.536481,716.991106,684.443317,457.472714,1016.427797,532.574969,513.629347,...,425.259988,441.646974,390.193471,545.998418,523.452413,533.302153,700.344297,417.067162,254.426683,344.767046
2026-12-31,367.494294,370.458293,324.565780,481.188381,730.890150,696.886359,466.803497,1036.054768,541.969138,525.364165,...,432.145340,449.573749,398.128787,560.125801,532.344791,541.777370,715.666902,424.478668,258.495415,350.569919
2027-12-31,373.543921,376.356748,329.601450,489.758451,744.426405,707.464407,474.267171,1055.441273,550.339283,536.877950,...,440.018740,458.874760,406.001573,572.702565,541.024076,551.714640,732.013331,431.998752,262.582762,356.444953
2028-12-31,379.558499,382.085234,334.623679,498.620725,758.040377,716.226951,483.051174,1074.756773,559.563898,548.511149,...,447.942533,468.178097,413.781946,586.766046,549.856193,560.991369,750.544855,439.367984,266.508151,362.294880
